# Diabetic Retinopathy Visualization Experiments

Visualizing the sample images from the [Kaggle competition](https://www.kaggle.com/c/diabetic-retinopathy-detection/data).

One [participant](https://github.com/hoytak/diabetic-retinopathy-code) pre-processed the images using ImageMagick. I've adopted his ideas for pre-processing, and added a few of my own.

In [ ]:
%matplotlib inline

import typing

from os.path import join, exists, isdir, isfile, split

import numpy as np
from PIL.Image import fromarray

from skimage.io import imread
from skimage.feature import canny
from skimage.color import rgb2gray

from scipy import ndimage as ndi

from IPython.display import Image, display

import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib
matplotlib.style.use('ggplot')

In [ ]:
im = imread('sample/10_left.jpeg')
im1 = imread('sample/10_left_conv.jpeg')
im2 = imread('sample/10_left_conv_2.jpeg')
im_512 = imread('sample/10_left_512_conv.jpeg')
msg = 'Image 10 {}'

## Image processing of the sample files

Let's look at the original image first.

In [ ]:
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(20,20))
ax.imshow(im)
ax.set_title(msg.format('Unprocessed'), y=1.05)
ax.xaxis.tick_top()
plt.imshow(im);


### Scaled to $256\times256$

Now let's look at two processed images. I used the ```convert``` command-line program that comes with ImageMagick.

```bash
convert -fuzz 10% -trim +repage -resize 256x256 -gravity center -background black -extent 256x256 -equalize 10_left.jpeg 10_left_conv.jpeg
```

Recall that the images are a tensor of size ```(3168, 4752, 3)```, for example. The data structure is a tensor since it has a third dimension, the *channel*, which is red, green, or blue. For each channel, the matrix has entries from 0 to 255, indicating the color.

The ```fuzz``` option has the effect that colors within this distance are considered equal.

The black border has also been stripped, and the image size is scaled to $256\times256$.

In [ ]:
fig, ax = plt.subplots(ncols=2, nrows=1, figsize=(15,15))
ax[0].imshow(im1);
ax[0].set_title(msg.format('\nprocessed with "-fuzz 10%"'), y=1.05)
ax[0].xaxis.tick_top()
ax[1].imshow(im2);
ax[1].set_title(msg.format('\nprocessed with "-fuzz 2%"'), y=1.05)
ax[1].xaxis.tick_top()
plt.tight_layout()

This is interesting, but let's compare each processed image to the unprocessed one.

In [ ]:
fig, ax = plt.subplots(ncols=2, nrows=1, figsize=(15,15))
ax[0].imshow(im);
ax[0].set_title(msg.format('\nunprocessed'), y=1.05)
ax[0].xaxis.tick_top()
ax[0].add_artist(patches.Rectangle((2000, 2000), 2000, 1000, alpha=2, fill=False, edgecolor='magenta'))
ax[1].imshow(im1);
ax[1].set_title(msg.format('\nprocessed with "-fuzz 10%"'), y=1.05)
ax[1].xaxis.tick_top()
ax[1].add_artist(patches.Rectangle((150, 200), 50, 50, alpha=2, fill=False,
                                  edgecolor='magenta'))
plt.tight_layout()

Other than the obvious size difference, we can see the blood vessels are brighter in the processed image, and we can also see a blood vessel in the lower right of the processed image that doesn't appear in the original. Also, the image overall is a brighter.

In [ ]:
fig, ax = plt.subplots(ncols=2, nrows=1, figsize=(15,15))
ax[0].imshow(im);
ax[0].set_title(msg.format('\nunprocessed'), y=1.05)
ax[0].xaxis.tick_top()
ax[0].add_artist(patches.Rectangle((2000, 2000), 2000, 1000, alpha=2, fill=False, edgecolor='magenta'))
ax[1].imshow(im2);
ax[1].set_title(msg.format('\nprocessed with "-fuzz 2%"'), y=1.05)
ax[1].xaxis.tick_top()
ax[1].add_artist(patches.Rectangle((150, 200), 50, 50, alpha=2, fill=False,
                                  edgecolor='magenta'))
plt.tight_layout()

There does not seem to be much difference between 10% and 2%.

### Rescaling the Image to $512 \times 512$

The image is bigger, of course, but it's also more expensive to process.

In [ ]:
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(20,20))
ax.imshow(im_512)
ax.set_title(msg.format('processed "fuzz 10%" 512x512'), y=1.05)
ax.xaxis.tick_top()
plt.imshow(im_512);

## Viewing All the Samples

We can see all the sample images using the following code. Just run

```python
show_eyes()
```

There are several obvious things here.

1. Some images do not have a notch, so they may be inverted.
2. Some images may be clipped at the top or bottom, or left or right.
3. An image may be blurred.

In [ ]:
def load_left_right(i: int, as_grey: bool=False) -> typing.Tuple[np.ndarray, np.ndarray]:
    left_file_name = join('sample', '{}_left.jpeg'.format(i))
    right_file_name = join('sample', '{}_right.jpeg'.format(i))
    im_l = imread(left_file_name)
    im_r = imread(right_file_name)
    if as_grey:
        return rgb2gray(im_l), rgb2gray(im_r)
    return im_l, im_r

In [ ]:
def side_by_side(im_l, im_r, cmap=None) -> None:
    fig, ax = plt.subplots(ncols=2, nrows=1, figsize=(20,10))
    ax[0].imshow(im_l, cmap=cmap)
    ax[0].set_title('Left eye', y=1.05)
    ax[0].xaxis.tick_top()
    ax[1].imshow(im_r, cmap=cmap)
    ax[1].set_title('Right eye', y=1.05)
    ax[1].xaxis.tick_top()

In [ ]:
def show_eyes() -> None:
    img_idx = [10, 13, 15, 16, 17]
    for i in img_idx:
        im_l, im_r = load_left_right(i)
        side_by_side(im_l, im_r)

In [ ]:
show_eyes()

## Viewing the Channels

Let's take a look at each channel of the unprocessed image.

In [ ]:
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(20,20))
ax.imshow(im[:, :, 0])
ax.set_title(msg.format('Unprocessed red channel'), y=1.05)
ax.xaxis.tick_top()
plt.imshow(im[:, :, 0]);

In [ ]:
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(20,20))
ax.imshow(im[:, :, 1])
ax.set_title(msg.format('Unprocessed green channel'), y=1.05)
ax.xaxis.tick_top()
plt.imshow(im[:, :, 1]);

In [ ]:
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(20,20))
ax.imshow(im[:, :, 2])
ax.set_title(msg.format('Unprocessed blue channel'), y=1.05)
ax.xaxis.tick_top()
plt.imshow(im[:, :, 2]);

<hr>

## Edge Detection

Let's use the [Canny edge detection algorithm](https://en.wikipedia.org/wiki/Canny_edge_detector) to see what we get. In this case, it's easier to work with grayscale images to find the boundary.

In [ ]:
im_l, im_r = load_left_right(10, as_grey=True)

In [ ]:
side_by_side(im_l, im_r, cmap='gray')

In [ ]:
left_10_edges = canny(im_l)
fill_left_10 = ndi.binary_fill_holes(left_10_edges)
right_10_edges = canny(im_r)
fill_right_10 = ndi.binary_fill_holes(right_10_edges)

Canny edge detection works well in this situation, at least for this example. Of course, we need to examine all the images.

In [ ]:
side_by_side(fill_left_10, fill_right_10, cmap='gray')

In [ ]:
plt.imshow(left_10_edges, cmap='gray')